In [1]:
from z3 import *

In [2]:
# Função para criar um LFSR com tamanho `n` e constante de feedback `s`
def create_lfsr(n, feedback_constant):
    # Cria um vetor de bits de tamanho `n` para representar o estado do LFSR
    state = BitVec('state', n)
    
    # Define o bit de saída e o próximo estado usando XOR e shift
    next_state = (state << 1) ^ feedback_constant
    output_bit = Extract(n-1, n-1, state)  # Pega o bit mais significativo
    
    return state, next_state, output_bit


In [3]:
# Constantes de feedback dos três LFSRs
s0 = 0b1110010000000000000   # Para LFSR de 19 bits
s1 = 0b1100000000000000000000 # Para LFSR de 22 bits
s2 = 0b11100000000010000000000 # Para LFSR de 23 bits

# Inicializando os LFSRs
state0, next_state0, output0 = create_lfsr(19, s0)
state1, next_state1, output1 = create_lfsr(22, s1)
state2, next_state2, output2 = create_lfsr(23, s2)


In [4]:
# Função para pegar o clocking bit
def get_clocking_bit(state, bit_position):
    return Extract(bit_position, bit_position, state)  # Extrai o bit da posição específica


In [5]:
clock_bit0 = get_clocking_bit(state0, 8)
clock_bit1 = get_clocking_bit(state1, 10)
clock_bit2 = get_clocking_bit(state2, 10)


In [6]:
# Regra de controle usando o "clocking bit" de cada LFSR
c0 = Or(clock_bit0 == clock_bit1, clock_bit0 == clock_bit2)
c1 = Or(clock_bit1 == clock_bit0, clock_bit1 == clock_bit2)
c2 = Or(clock_bit2 == clock_bit0, clock_bit2 == clock_bit1)


In [7]:
# Calcula o bit de chave a partir dos bits de saída dos LFSRs
key_bit = output0 ^ output1 ^ output2


In [ ]:
# Função para avançar o estado do LFSR condicionalmente
def advance_lfsr(state, next_state, control_bit):
    # Se o bit de controle for 1, avançamos o estado
    return If(control_bit == 1, next_state, state)


In [ ]:
# Avançando o estado de cada LFSR de acordo com os bits de controle
new_state0 = advance_lfsr(state0, next_state0, c0)
new_state1 = advance_lfsr(state1, next_state1, c1)
new_state2 = advance_lfsr(state2, next_state2, c2)
